In [1]:
ids = addprocs(4) # currently works only with a square number of processes

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
push!(LOAD_PATH, pwd())

3-element Array{Any,1}:
 "/home/stefan/looseends/julia-903644385b/local/share/julia/site/v0.6"
 "/home/stefan/looseends/julia-903644385b/share/julia/site/v0.6"      
 "/home/stefan/python_lab35/resource_overhead/qevo_3"                 

In [3]:
include("Clifford.jl")

Clifford

In [4]:
@everywhere using Clifford

In [5]:
@everywhere using IterTools
@everywhere using Combinatorics

In [50]:
@everywhere begin
d_bellstatestabs = Dict(:A=>(p"+XX",p"+ZZ"),
                        :B=>(p"-XX",p"-ZZ"),
                        :C=>(p"+XX",p"-ZZ"),
                        :D=>(p"-XX",p"+ZZ"))

function bellstate_stabilizers(state::Symbol)
    d_bellstatestabs[state]
end

bellpair_basis = collect(values(d_bellstatestabs))
pair1 = [[expand_2toN(4,1,3,op) for op in s] for s in bellpair_basis]
pair1_complete = [[a, b, a*b] for (a,b) in pair1]
pair2 = [[expand_2toN(4,2,4,op) for op in s] for s in bellpair_basis]
pair2_complete = [[a, b, a*b] for (a,b) in pair2]
twopairs_basis = [[p1 ; p2] for (p1, p2) in product(pair1,pair2)]
twopairs_basis_complete = [[p1 ; p2] for (p1, p2) in product(pair1_complete,pair2_complete)]
twopairs_basis_set = Set(Set(_) for _ in twopairs_basis);
twopairs_basis_set_complete = Set(Set(_) for _ in twopairs_basis_complete);

preserves(op::CliffordOperator) = (map(s->map(p->op*p,
                                              s),
                                       twopairs_basis_set)
                                   ==twopairs_basis_set)

twopairs_basis_set_hashesums = sort([sum([hash(s) for s in stabs]) 
                                     for stabs in twopairs_basis_set])

#function preserves_hashsum(op::CliffordOperator)
#    result_hashsums = Array{UInt64,1}(16)
#    for i in 1:16
#        result_hashsums[i] = (hash(op*twopairs_basis[i][1])
#                             +hash(op*twopairs_basis[i][2])
#                             +hash(op*twopairs_basis[i][3])
#                             +hash(op*twopairs_basis[i][4]))
#    end
#    sort!(result_hashsums)
#    return result_hashsums==twopairs_basis_set_hashesums
#end

function preserves_hashsum(op::CliffordOperator)
    result_hashsums = Array{UInt64,1}(16)
    ptemp = p"IIII"
    for i in 1:16
        begin
            result_hashsums[i] = 0
            for j in 1:4
                mul!(op, twopairs_basis[i][j], ptemp)
                result_hashsums[i] += hash(ptemp)    
            end
        end
    end
    sort!(result_hashsums)
    return result_hashsums==twopairs_basis_set_hashesums
end

c_id = reduce(⊗, [CliffordOperator([p"Z"], [p"X"]) for i in 1:4])

end

In [9]:
@everywhere begin
C = collect(clifford_group(2))
function get_sub_iterator(i, n)
    l = isqrt(n)
    @assert l^2==n
    j, k = divrem(i-1,l)
    len = div(length(C),l)
    if j==l-1 && k==l-1
        product(C[j*len+1:end],C[k*len+1:end])
    elseif j==l-1
        product(C[j*len+1:end],C[k*len+1:(k+1)*len])
    elseif k==l-1
        product(C[j*len+1:(j+1)*len],C[k*len+1:end])
    else
        product(C[j*len+1:(j+1)*len],C[k*len+1:(k+1)*len])
    end
end

function work(p,n,report_array)
    report_array[p] = 0
    bell_diagonal_op = Array{CliffordOperator, 1}()
    for (o1,o2) in get_sub_iterator(p,n)
        op = o1 ⊗ o2
        if preserves_hashsum(op) && preserves(op)
            push!(bell_diagonal_op, op)
        end
        report_array[p] += 1
    end
    bell_diagonal_op
end

end

In [42]:
n = length(workers())
results = Array{Any, 1}(n)
report_array = SharedArray{Int}(n)
for (i,p) in enumerate(workers())
    @async begin results[i] = remotecall_fetch(work, p, i, n, report_array) end
end

In [175]:
sum(report_array) - length(product(C,C))

0

In [176]:
sum(length(_) for _ in results)

2048

In [177]:
11520/2048

5.625